In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, PeftConfig

In [2]:
json_data = {"input": "请根据查询和给出的参考文档生成答案回答问句并添加合适的引用。输入查询:为什么我们会掉牙？为什么我们有乳牙？参考文章:[1]：当婴儿逐渐的长大，骨骼也在慢慢的变宽，这时如果还是原来的20颗乳牙，就很容易造成牙齿间的缝隙过大，影响食物的摄取，而且由于食物类型的转变，也需要更加坚硬的牙齿，所以恒牙的牙釉质较之乳牙是更厚的，而且恒牙的数量更多，可达到28-32颗。\n[2]：牙齿之所以要脱落才能完成替换，是因为牙齿上没有骨膜，一般骨头的生长是由于骨膜的包裹，才能将骨骼在原来的基础上增厚，但是牙齿却没有，所以必须通过脱落的方式，才能重新长出新的牙齿。\n", "output": "牙齿之所以要脱落才能完成替换，是因为牙齿上没有骨膜，一般骨头的生长是由于骨膜的包裹，才能将骨骼在原来的基础上增厚，但是牙齿却没有，所以必须通过脱落的方式，才能重新长出新的牙齿。乳牙就相当于是恒牙的临时替代品，小时候的食物一般是流质性或者较软糯的食物，再加上一生下来就必须进食，所以这时快速生长出来的乳牙都是较为脆弱的，而且婴儿的骨骼较小，所以总共只有20颗左右，是不能满足成熟之后的进食需求的。当婴儿逐渐的长大，骨骼也在慢慢的变宽，这时如果还是原来的20颗乳牙，就很容易造成牙齿间的缝隙过大，影响食物的摄取，而且由于食物类型的转变，也需要更加坚硬的牙齿，所以恒牙的牙釉质较之乳牙是更厚的，而且恒牙的数量更多，可达到28-32颗。【1】【2】【3】"}

In [3]:
print(json_data)

{'input': '请根据查询和给出的参考文档生成答案回答问句并添加合适的引用。输入查询:为什么我们会掉牙？为什么我们有乳牙？参考文章:[1]：当婴儿逐渐的长大，骨骼也在慢慢的变宽，这时如果还是原来的20颗乳牙，就很容易造成牙齿间的缝隙过大，影响食物的摄取，而且由于食物类型的转变，也需要更加坚硬的牙齿，所以恒牙的牙釉质较之乳牙是更厚的，而且恒牙的数量更多，可达到28-32颗。\n[2]：牙齿之所以要脱落才能完成替换，是因为牙齿上没有骨膜，一般骨头的生长是由于骨膜的包裹，才能将骨骼在原来的基础上增厚，但是牙齿却没有，所以必须通过脱落的方式，才能重新长出新的牙齿。\n', 'output': '牙齿之所以要脱落才能完成替换，是因为牙齿上没有骨膜，一般骨头的生长是由于骨膜的包裹，才能将骨骼在原来的基础上增厚，但是牙齿却没有，所以必须通过脱落的方式，才能重新长出新的牙齿。乳牙就相当于是恒牙的临时替代品，小时候的食物一般是流质性或者较软糯的食物，再加上一生下来就必须进食，所以这时快速生长出来的乳牙都是较为脆弱的，而且婴儿的骨骼较小，所以总共只有20颗左右，是不能满足成熟之后的进食需求的。当婴儿逐渐的长大，骨骼也在慢慢的变宽，这时如果还是原来的20颗乳牙，就很容易造成牙齿间的缝隙过大，影响食物的摄取，而且由于食物类型的转变，也需要更加坚硬的牙齿，所以恒牙的牙釉质较之乳牙是更厚的，而且恒牙的数量更多，可达到28-32颗。【1】【2】【3】'}


In [4]:
peft_model_path = '/mnt/workspace/chatGLM-6B-QLoRA/saved_files/chatGLM_6B_QLoRA_t32'


config = PeftConfig.from_pretrained(peft_model_path)
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=torch.float32)

base_model = AutoModel.from_pretrained(config.base_model_name_or_path,
                                       quantization_config=q_config,
                                       trust_remote_code=True,
                                       device_map='auto')

input_text = json_data['input']
print(f'输入：\n{input_text}')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path, trust_remote_code=True)

response, history = base_model.chat(tokenizer=tokenizer, query=input_text)
print(f'微调前：\n{response}')

model = PeftModel.from_pretrained(base_model, peft_model_path)
response, history = model.chat(tokenizer=tokenizer, query=input_text)
print(f'微调后: \n{response}')

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

输入：
请根据查询和给出的参考文档生成答案回答问句并添加合适的引用。输入查询:为什么我们会掉牙？为什么我们有乳牙？参考文章:[1]：当婴儿逐渐的长大，骨骼也在慢慢的变宽，这时如果还是原来的20颗乳牙，就很容易造成牙齿间的缝隙过大，影响食物的摄取，而且由于食物类型的转变，也需要更加坚硬的牙齿，所以恒牙的牙釉质较之乳牙是更厚的，而且恒牙的数量更多，可达到28-32颗。
[2]：牙齿之所以要脱落才能完成替换，是因为牙齿上没有骨膜，一般骨头的生长是由于骨膜的包裹，才能将骨骼在原来的基础上增厚，但是牙齿却没有，所以必须通过脱落的方式，才能重新长出新的牙齿。

微调前：
我们的牙齿会掉牙是因为婴儿逐渐长大，骨骼变宽，如果牙齿数量没有跟随长大，就会导致牙齿间的缝隙过大，影响食物摄入。同时，食物类型的转变也会需要更加坚硬的牙齿。恒牙的牙釉质较之乳牙更厚，且恒牙的数量更多，可达到28-32颗。

牙齿之所以需要脱落才能完成替换，是因为牙齿上没有骨膜。一般骨头的生长是由于骨膜的包裹，才能将骨骼在原来的基础上增厚。但是牙齿却没有，所以必须通过脱落的方式，才能重新长出新的牙齿。
微调后: 
因为婴儿逐渐的长大，骨骼也在慢慢的变宽，这时如果还是原来的20颗乳牙，就很容易造成牙齿间的缝隙过大，影响食物的摄取，而且由于食物类型的转变，也需要更加坚硬的牙齿，所以恒牙的牙釉质较之乳牙是更厚的，而且恒牙的数量更多，可达到28-32颗。【1】


In [6]:
input_str="""
请根据查询和给出的参考文档生成答案回答问句并添加合适的引用。输入查询:像麦当劳这样的公司是如何保证他们所有的店的食物味道完全相同？ 无论何时何地，味道总是一样的。参考文章:[1]：一、高效率背后的训练系统麦当劳化的系统是理性和高效的。那么问题来了，为这个系统服务的员工，和为这个系统贡献时间与金钱的消费者，他们那些基于人性的诉求究竟能于何处安放？从流程上来看，麦当劳的高效率是基于后台高度智能设备化+中台强大的人力训练系统，提供给前台顾客人性化的服务和产品。设备解决的是产品标准化并提升效率，解放了员工的劳动力去做更多有价值的顾客服务。所以，某种意义上来说，设备化是麦当劳标准化的重要犄角之一，这点是很多人忽略的。\n[2]：1、以人为本的麦当劳在麦当劳的黄金拱门餐厅里，顾客除了可以享受到最快的餐饮，同时还能享受到人性化的服务,而这正是麦当劳 “提供全世界最卓越的快速服务餐厅经验” 的愿景。「人员」「顾客」「组织成长」是麦当劳达成愿景的三大策略, 而「人员」更是麦当劳最最重要的资产，麦当劳的产品是经由「人」传递给顾客的，所以麦当劳是个非常重视「人」的事业。2、麦当劳全球化学习发展系统，『全球品牌，社区经营』的最佳写照 － 香港汉堡大学麦当劳企业在1955年开始营运后,即于1961年，选择了当时刚落成的伊利诺州Elk Grove村的麦当劳餐厅，开始了汉堡大学的培训课程。随着国际市场的需求愈来愈大，麦当劳为了更有效率地培训全球国际化人才，开始于各区域设立国际汉堡大学，目前全球已有七所，分别位于德国、巴西、澳洲、日本、美国、英国、香港。麦当劳是一个庞大的家庭，全球210个国家中已有超过三万家麦当劳餐厅，而截至目前为止，汉堡大学已拥有超过七万名高级营运课程的毕业生，另外则还有数千名的高阶主管获得这所世界级的汉堡大学其它课程的结业证书。所谓「全球品牌,地区经营」香港汉堡大学的成立便是最佳的写照。为了因应华人市场的快速发展，包括大陆有五百余家，台湾三百五十余家，以及香港、澳门的二百余家麦当劳餐厅 ; 麦当劳特于公元二千年在香港成立汉堡大学，针对华人地区的麦当劳主管进行人才培训工作。汉堡大学的训练课程，是针对餐厅经理及以上的中、高阶主管所设计。并针对储备经理人才设计一系列的生涯规划。当麦当劳员工的职级在麦当劳担任到餐厅经理时，就代表此员工已有足够的能力管理一家资产及营业额达到数千万元的餐厅。为培训管理人才有国际观的视野，特别安排餐厅经理以上的员工到汉堡大学接受训练，以提升整体经营水准。香港汉堡大学的成立有自己的既定目标; 包含了发展更符合地区市场的课程内容,避免因为语言及时差上造成的学习障碍以及更有效的运用培训经费,极大化整体的训练成效。3、香港汉堡大学的策略计划愿景：成为最佳之人员培训专家，致力于麦当劳核心职能及领导职能之提升以达成麦当劳全球愿景。\n
"""

In [16]:
response, history = model.chat(tokenizer=tokenizer, query=input_str)
print(f'微调后: \n{response}')

微调后: 
中国人寿的市占率保持在行业第一。作为国内寿险业的龙头，中国人寿的市占率虽然在2008~2017年出现大幅下滑，但仍占据行业第一。市占率下滑一方面是由于近年来中小寿险企业的迅速崛起，市场充分发挥个性化优势，向多元化方向发展，挤压行业内市场份额；另一方面，中国人寿加强战略调整，主动优化结构、控制银保趸交业务规模，使得市场份额逐年下滑，最低滑落至仅为23%。2018年市占率企稳回升至25%左右，后续年份稳步上升，战略转型和调整展现出了良好的效果。【1】


In [8]:
input_str = '请根据查询和给出的参考文档生成答案回答问句并添加合适的引用。输入查询:为什么url中用%20来表示一个空格?参考文章:[1]：一个URL的基本组成部分包括协议(scheme)、 域名、端口号、路径和查询字符串（路径参数和锚点标记就暂不考虑了）。路径和查询字符串之间用问号?分离。例如http://www.example.com/index?param=1，路径为index，查询字符串(Query String)为param=1。URL中关于空格的编码正是与空格所在位置相关：空格被编码成加号+的情况只会在查询字符串部分出现，而被编码成%20则可以出现在路径和查询字符串中。'

In [10]:
input_str ="""请根据查询和给出的参考文档生成答案回答问句并添加合适的引用。输入查询:为什么手湿时按压手机屏幕没反应？参考文章:[1]：淋水后出现屏幕失灵现象的大多都是采用电容屏的手机，电容屏是利用人体的电流感应进行工作。\n[2]：而当屏幕被水淋湿后，因为水的可导电特性，当水接触到屏幕，手再去摸，电流就会被影。\n[3]：向大家说明下，其实手机屏幕分为电容屏以及电阻屏，目前淋水后出现屏幕失灵现象的大多都是采用电容屏的手机，电容屏是利用人体的电流感应进行工作。其触摸屏四边均镀上了狭长的电极，当用户触碰屏幕，由于人体电场，用户和触摸屏表面会形成一个耦合电容，对于高频电流来说，电容是直接导体，于是手指就会从接触点吸走一个很小的电流。这个电流分从触摸屏的四角上的电极中流出，并且流经这四个电极的电流与手指到四角的距离成正比，控制器通过对这四个电流比例的精确计算，得出触摸点的位置。"""


In [12]:
input_str="""请根据查询和给出的参考文档生成答案回答问句并添加合适的引用。输入查询:宝钢股份主营产品的市占率是多少?参考文章:[1]:公司主营板材，2019年公司板材（冷轧、热轧板）产量占比高达82.3%；板材（冷轧、热轧板）业务收入占比钢铁业务营收比例为80.3%，占比总营收55.4%。根据中钢协数据，2019年全行业汽车板、家电板产量合计近3000万吨，公司汽车板、家电板产量合计近1900万吨，占全行业产量的比例为63%。\n[2]:1.压缩产量+碳排放权配额钢铁行业是碳排放的重要领域，钢铁行业低碳减排对全国实现碳达峰的目标至关重要。目前我国钢铁行业低碳减排政策主要有两种：一是压缩钢铁产量。二是实施碳排放权限额分配制度。"""

In [15]:
input_str="""请根据查询和给出的参考文档生成答案回答问句并添加合适的引用。输入查询:中国人寿的市占率是多少?参考文章:[1]:中国人寿市占率持续占据行业第一。作为国内寿险业的龙头，中国人寿的市占率虽然在2008~2017年出现大幅下滑，但仍占据行业第一。市占率下滑一方面是由于近年来中小寿险企业的迅速崛起，市场充分发挥个性化优势，向多元化方向发展，挤压行业内市场份额；另一方面，中国人寿加强战略调整，主动优化结构、控制银保趸交业务规模，使得市场份额逐年下滑，最低滑落至仅为23%。2018年市占率企稳回升至25%左右，后续年份稳步上升，战略转型和调整展现出了良好的效果。\n[2]:中国人寿的市占率虽然在2008~2017年出现大幅下滑，但仍占据行业第一。"""